In [2]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import math
import gensim
global df, show_descriptions

meta_data = []
with open("../data/metadata.tsv") as csvfile:
    csvreader = csv.reader(csvfile,delimiter="\t")
    for row in csvreader:
        meta_data.append(row)

df = pd.DataFrame(meta_data[1:],columns=meta_data[0])
show_filename_prefixes = df.show_filename_prefix
episode_filename_prefixes = df.episode_filename_prefix
show_names = df.show_name
episode_names = df.episode_name
show_duration = df.duration
publisher_name = df.publisher

show_descriptions = {}

for item1,item2 in zip(list(show_names),list(df.show_description)):
    show_descriptions[item1] = item2.strip()
    
episode_descriptions = {}
for item1,item2 in zip(list(episode_filename_prefixes),list(df.episode_description)):
    episode_descriptions[item1] = item2.strip()   

In [29]:
transcripts = {}
for k,showname in enumerate(show_filename_prefixes[:100000]):
    s = showname.split("_")[1]
    try:
        with open('../podcast_transcripts/'+s[0]+'/'+s[1]+'/'+showname+'/'+episode_filename_prefixes[k]+'.txt','r') as f:
            transcripts[(showname,episode_filename_prefixes[k])] = f.readlines()
        f.close()
    except Exception as e:
        pass
    
keys = list(transcripts.keys())

In [30]:
def normalize_docs(text_data):
    final_texts = []
    for text in text_data:
        new_text = gensim.utils.simple_preprocess(text,deacc=True)
        final_texts.append(new_text)
    return final_texts

In [31]:
docs = []
for text in transcripts:
    aggregated = []
    for words in normalize_docs(transcripts[keys[0]]):
        aggregated.extend(words)
    docs.append(aggregated)

In [32]:
model=gensim.models.Word2Vec(
    window = 10,
    min_count = 2,
    workers = 10
)

In [33]:
model.build_vocab(docs,progress_per=1000)

In [34]:
model.corpus_count

58452

In [35]:
model.train(docs,total_examples=model.corpus_count,epochs=model.epochs)

(401924427, 596210400)

In [46]:
model.wv.similarity(w1="good",w2="great")

-0.09149644